In [1]:
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut
from gc_utils import get_descendants_halt, iteration_name, main_prog_halt, snapshot_name  # type: ignore


In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

snap_dir = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap_dir, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

snap_pub_dir = sim_dir + "/snapshot_times_public.txt"
snap_pub_data = pd.read_table(snap_pub_dir, comment="#", header=None, sep=r"\s+")
snap_pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

Get COM, and SIGMA of GC system in relation to its host galaxy (pre accretion) and then the MW (post accretion)

In [44]:
# halt = halo.io.IO.read_tree(simulation_directory=fire_dir)
halt = halo.io.IO.read_tree(simulation_directory=fire_dir)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

* read 17859038 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5



In [4]:
def get_host_at_snap(grp_val, snap_final, acc_snap, halt):
    if grp_val != 0:
        idx = np.where(halt["tid"] == grp_val)[0][0]
        snap = halt["snapshot"][idx]

        if snap < acc_snap:
            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

        else:
            idx = 0
            snap = halt["snapshot"][idx]

            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

    else:
        idx = 0
        snap = halt["snapshot"][idx]

        while snap > snap_final:
            idx = halt["progenitor.main.index"][idx]
            snap = halt["snapshot"][idx]

    host_tid = halt["tid"][idx]
    host_idx = idx

    # print(snap, host_tid)

    return host_tid, host_idx

In [5]:
def get_acc_snap(halo_tid, main_halo_tid, halt):
    tid_main_lst = main_prog_halt(halt, main_halo_tid)
    desc_lst = get_descendants_halt(halo_tid, halt)

    idx_lst = np.array([1 if halt["tid"][idx] in tid_main_lst else 0 for idx in desc_lst])
    idx_acc = np.where(idx_lst == 1)[0][0]

    snap_acc = halt["snapshot"][desc_lst[idx_acc]]

    return snap_acc

In [28]:
snap = 77

part = gizmo.io.Read.read_snapshots(
    ["dark", "star"],
    "index",
    snap,
    simulation_directory=fire_dir,
)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  r

In [68]:
halt["position"][2105151]

array([43402.766, 41943.22 , 44336.91 ], dtype=float32)

In [69]:
halt.prop("host.distance.principle", 2105151)

array([0., 0., 0.], dtype=float32)

In [70]:
dist_vect = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=part.host["position"],
    rotation=None,
)

In [71]:
dist_vect["dark"][0]

array([ 57.04168656, -33.88838981,   8.97484625])

In [72]:
part["dark"]["position"][0] - part.host["position"]

array([[ 312.48921191, -185.6494235 ,   49.16654469]])

In [73]:
part["dark"].prop("host.distance.principle", 0)

array([ 57.04168656, -33.88838981,   8.97484625])

In [80]:
vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=part.host["position"],
    center_velocity=part.host["velocity"],
    rotation=None,
)

In [81]:
vel_vect["dark"][0]

array([-86.061935 ,   2.9947548, -30.712414 ], dtype=float32)

In [82]:
part["dark"].prop("host.velocity.principle", 0)

array([-86.061935 ,   2.9947548, -30.712414 ], dtype=float32)

In [83]:
np.linalg.norm(vel_vect["dark"][0])

91.4269

In [84]:
np.linalg.norm(part["dark"].prop("host.velocity.principle", 0))

91.4269

THIS LOOKS GOOD FINN

In [256]:
hals = halo.io.IO.read_catalogs("index", [77, 600], fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

* using snapshot indices = [ 77 600]
  redshifts = [4.47826052 0.        ]

* read 9832 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/halo_077.hdf5

# in halo_analysis.halo_io.Particle():
* read 9832 halos, 301 have star particles, from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/star_077.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/track/host_coordinates.hdf5
  host1 position = (43057.26, 42130.64, 44311.36) [kpc comoving]
  host1 velocity = (-11.0, 39.1, 70.4) [km/s]
  host1 axis ratios = (0.53, 0.65, 0.81)

* read 35838 halos from:  Users/z5114326/Documents/simula

In [165]:
part = gizmo.io.Read.read_snapshots(
    ["dark", "star"], "index", snap, simulation_directory=fire_dir, assign_hosts_rotation=True
)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  r

From part 77

host1 position = (43057.26, 42130.64, 44311.36) [kpc comoving]

In [166]:
snap = 77

host_idx = np.where(hals[snap].prop("host.distance.principle.total") == 0)[0][0]
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [178]:
dist_vect = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    rotation=None,
)

In [183]:
dist_vect["dark"][0]

array([ 57.23848718, -33.88696372,   9.04401168])

In [201]:
part["dark"]["position"][0] - hals[snap]["position"][host_idx]

array([ 313.56733691, -185.641611  ,   49.54545094])

In [ ]:
ut.coordinate.get_distances(
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

array([ 57.23848718, -33.88696372,   9.04401168])

In [204]:
ut.coordinate.get_distances(
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    hals[snap].info["box.length"],
    hals[snap].snapshot["scalefactor"],
    # total_distance,
)  # [kpc physical]

array([ 57.23858168, -33.88701967,   9.04402661])

In [182]:
part["dark"].prop("host.distance.principle", 0)

array([ 57.04168656, -33.88838981,   8.97484625])

In [206]:
# lets do it with galaxy centres

ut.coordinate.get_distances(
    hals[snap]["position"][10],
    hals[snap]["position"][host_idx],
    hals[snap].info["box.length"],
    hals[snap].snapshot["scalefactor"],
    # total_distance,
)  # [kpc physical]

array([-201.62328 ,   81.791466,  -79.352844], dtype=float32)

In [210]:
hals[snap].prop("host.distance.principle", 10)

array([-201.62328 ,   81.791466,  -79.352844], dtype=float32)

HOLY SHIT IT WORKS

In [245]:
vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    # center_velocity=hals[snap]["velocity"][host_idx],
    rotation=None,
)

In [246]:
vel_vect["dark"][0]

array([-85.9688   ,   2.9954295, -30.679682 ], dtype=float32)

In [278]:
part["dark"].prop("host.velocity.principal", 0)

array([-87.89803,  23.71265,   8.39786], dtype=float32)

In [279]:
part["dark"].prop("host.velocity.principle", 0)

array([-86.061935 ,   2.9947548, -30.712414 ], dtype=float32)

In [248]:
vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    center_velocity=hals[snap]["velocity"][host_idx],
    rotation=None,
)

vel_vect["dark"][0]

array([-118.61138 ,    6.050033,  -43.019585], dtype=float32)

In [249]:
ut.coordinate.get_velocity_differences(
    part["dark"]["velocity"][0],
    hals[snap]["velocity"][host_idx],
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    part.snapshot["time.hubble"],
    total_velocity=False,
)

array([-118.61138 ,    6.050033,  -43.019585], dtype=float32)

In [228]:
ut.coordinate.get_velocity_differences(
    part["dark"]["velocity"][0],
    part.host["velocity"],
    part["dark"]["position"][0],
    part.host["position"],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    part.snapshot["time.hubble"],
    total_velocity=False,
)

array([[-86.061935 ,   2.9947548, -30.712414 ]], dtype=float32)

IT LOOKS LIKE THERE IS A DIFFERENT IN HOST VELOCITY

In [229]:
np.linalg.norm(hals[snap]["velocity"][host_idx])

92.85362

In [230]:
np.linalg.norm(part.host["velocity"])

81.317474

WTF!!!!!!

In [233]:
hals[snap].host["velocity"]

array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32)

In [235]:
hals[snap]["velocity"][host_idx]

array([21.66, 36.08, 82.77], dtype=float32)

In [236]:
part.host["velocity"]

array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32)

In [237]:
hals[snap].host["position"]

array([[43057.26 , 42130.637, 44311.363]], dtype=float32)

In [238]:
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [239]:
part.host["position"]

array([[43057.26 , 42130.637, 44311.363]], dtype=float32)

In [265]:
print(hals[snap].host["velocity"] - hals[snap]["velocity"][host_idx])

[[-32.64258     3.0546036 -12.339905 ]]


In [270]:
ut.particle.get_center_velocities_or_accelerations(
    part,
    property_kind="velocity",
    species_name="star",
    part_indicess=None,
    weight_property="mass",
    distance_max=10,
    center_positions=None,
    return_single_array=True,
    verbose=True,
)


# in utilities.particle.get_center_velocities_or_accelerations():
* assigning velocity for 1 center/host, weighting star particles by mass
  host1 velocity = (-10.6, 39.8, 69.4) [km/s]


array([-10.581415,  39.823235,  69.449585], dtype=float32)

DONT FORGET WILL NEED TO DO VELOCITY DISPERSION